#### Setting environments

In [1]:
import os, itertools, gc
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score,\
f1_score, precision_score, recall_score, roc_auc_score, average_precision_score

In [2]:
# Set options
embed_ver = ["clstm", "esm2", "bert", "t5"]
test_ver = ["C018", "C039", "C035", "D011", "C048"]

result_path = f"../results/"
save_path = f"../results/prd-conf/"
os.makedirs(save_path, exist_ok=True)

col_str = ['file_id', 'organism', 'locus_tag', 'ess']

In [3]:
# Define function to record perfomance result
def record_perform(comb_ver, file_id, organ, y_real, y_conf, y_prd):    
    if file_id != "O046":
        auc_roc = [roc_auc_score(y_real, y_conf)]
        auc_pr = [average_precision_score(y_real, y_conf)]
    else:
        auc_roc = None
        auc_pr = None
    
    tn, fp, fn, tp = confusion_matrix(y_real, y_prd).ravel()
    
    result = pd.DataFrame({
        "comb": [comb_ver],
        "file": [file_id],
        "organism": [organ],
        "tp": [tp],
        "fp": [fp],
        "tn": [tn],
        "fn": [fn],
        "mcc": [matthews_corrcoef(y_real, y_prd)],
        "acc": [accuracy_score(y_real, y_prd)],
        "f1": [f1_score(y_real, y_prd)],
        "prc": [precision_score(y_real, y_prd)],
        "rec": [recall_score(y_real, y_prd)],
        "npv": [precision_score(1 - y_real, 1 - y_prd)],
        "tnr": [recall_score(1 - y_real, 1 - y_prd)],
        "auc-roc": auc_roc,
        "auc-pr": auc_pr
    })

    return result

#### Evaluate model

In [4]:
df_eval = pd.DataFrame()

for r in range(2, len(embed_ver) + 1):
    combs = list(itertools.combinations(embed_ver, r))
    
    for comb in combs:
        comb_ver = "_".join(comb)
        print(f"\n>>>> {comb_ver} <<<<")
        
        total_conf = []
        total_label = []
        total_cls = []
        for ts_ver in test_ver:
            dfs = [(e_ver, pd.read_csv(f"{result_path}prd-indiv/{e_ver}-{ts_ver}.csv")) for e_ver in comb]

            # merge dataset
            data = dfs[0][1]
            for df in dfs[1:]:
                data = pd.merge(data, df[1], on=col_str, suffixes=("", f"_{df[0]}"))
            
            display("Raw data:", data)
            
            # calculate mean of confidences
            col_num = [col for col in data.columns if col not in col_str]
            data['conf_mean'] = data[col_num].mean(axis=1)
            
            # save the model prediction result
            data.to_csv(f"{save_path}{comb_ver}-{ts_ver}.csv", index=False)

            
            ## evaluations by test dataset ##
            prd_cls = (data['conf_mean'] >= 0.5).astype(int).tolist()
            
            # gather the results
            total_conf.extend(data['conf_mean'].tolist())
            total_label.extend(data['ess'].tolist())
            total_cls.extend(prd_cls)

            # performances by testset
            eval_ts = record_perform(
                comb_ver=comb_ver,
                file_id=ts_ver,
                organ=ts_ver,
                y_real=data['ess'].to_numpy(),
                y_conf=data['conf_mean'].to_numpy(),
                y_prd=np.array(prd_cls)
            )
            df_eval = pd.concat([df_eval, eval_ts], ignore_index=True)
            print(f"- Test in {ts_ver} was done.")
            
            gc.collect()
            
        # performances on total testset
        eval_ts = record_perform(
            comb_ver=comb_ver,
            file_id="total",
            organ="all",
            y_real=np.array(total_label),
            y_conf=np.array(total_conf),
            y_prd=np.array(total_cls)
        )
        df_eval = pd.concat([df_eval, eval_ts], ignore_index=True)
        print(f"- Test in total testset was done.")

# save the model perfomance result
display("Model performance:", df_eval)
df_eval.to_csv(f"{result_path}eval-conf_avg-strain.csv", index=False)


>>>> clstm_esm2 <<<<


'Raw data:'

,file_id,organism,locus_tag,ess,conf,conf_esm2
0,C018,Escherichia coli K-12 BW25113,BW25113_0001,0,0.154492,0.010318
1,C018,Escherichia coli K-12 BW25113,BW25113_0002,0,0.200179,0.605639
2,C018,Escherichia coli K-12 BW25113,BW25113_0003,0,0.057514,0.100957
3,C018,Escherichia coli K-12 BW25113,BW25113_0004,0,0.244505,0.522194
4,C018,Escherichia coli K-12 BW25113,BW25113_0005,0,0.121918,0.094325
...,...,...,...,...,...,...
4308,C018,Escherichia coli K-12 BW25113,BW25113_4702,0,0.458123,0.023436
4309,C018,Escherichia coli K-12 BW25113,BW25113_4703,0,0.043490,0.165554
4310,C018,Escherichia coli K-12 BW25113,BW25113_4705,0,0.056452,0.100853
4311,C018,Escherichia coli K-12 BW25113,BW25113_4706,0,0.040557,0.225901


- Test in C018 was done.


'Raw data:'

...

'Raw data:'

,file_id,organism,locus_tag,ess,conf,conf_esm2,conf_bert,conf_t5
0,C048,Bacteroides thetaiotaomicron VPI-5482,BT_0001,0,0.042788,0.083858,0.079640,0.057938
1,C048,Bacteroides thetaiotaomicron VPI-5482,BT_0002,0,0.063893,0.005515,0.149323,0.128610
2,C048,Bacteroides thetaiotaomicron VPI-5482,BT_0003,0,0.037608,0.096812,0.110480,0.038551
3,C048,Bacteroides thetaiotaomicron VPI-5482,BT_0004,1,0.039393,0.575127,0.589309,0.561807
4,C048,Bacteroides thetaiotaomicron VPI-5482,BT_0005,0,0.048480,0.610858,0.633530,0.691487
...,...,...,...,...,...,...,...,...
4820,C048,Bacteroides thetaiotaomicron VPI-5482,BT_p548234,0,0.081106,0.062600,0.090554,0.128954
4821,C048,Bacteroides thetaiotaomicron VPI-5482,BT_p548235,0,0.031723,0.264326,0.433809,0.218671
4822,C048,Bacteroides thetaiotaomicron VPI-5482,BT_p548236,0,0.141442,0.157561,0.306851,0.066294
4823,C048,Bacteroides thetaiotaomicron VPI-5482,BT_p548237,0,0.027046,0.328888,0.442205,0.207544


- Test in C048 was done.
- Test in total testset was done.


'Model performance:'

,comb,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,clstm_esm2,C018,C018,260,206,3809,38,0.670862,0.943427,0.680628,0.557940,0.872483,0.990122,0.948692,0.944941,0.746249
1,clstm_esm2,C039,C039,256,202,5017,95,0.610901,0.946679,0.632880,0.558952,0.729345,0.981416,0.961295,0.964949,0.711951
2,clstm_esm2,C035,C035,307,188,3218,173,0.576736,0.907102,0.629744,0.620202,0.639583,0.948983,0.944803,0.925803,0.719834
3,clstm_esm2,D011,D011,194,288,2321,89,0.458522,0.869640,0.507190,0.402490,0.685512,0.963071,0.889613,0.866100,0.535826
4,clstm_esm2,C048,C048,141,386,4114,184,0.279688,0.881865,0.330986,0.267552,0.433846,0.957189,0.914222,0.761322,0.308442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,clstm_esm2_bert_t5,C039,C039,306,204,5015,45,0.701583,0.955296,0.710801,0.600000,0.871795,0.991107,0.960912,0.975487,0.792919
62,clstm_esm2_bert_t5,C035,C035,380,190,3216,100,0.684395,0.925373,0.723810,0.666667,0.791667,0.969843,0.944216,0.946106,0.804883
63,clstm_esm2_bert_t5,D011,D011,216,245,2364,67,0.543297,0.892116,0.580645,0.468547,0.763251,0.972439,0.906094,0.912770,0.658301
64,clstm_esm2_bert_t5,C048,C048,194,429,4071,131,0.374908,0.883938,0.409283,0.311396,0.596923,0.968824,0.904667,0.792085,0.403253
